<a href="https://colab.research.google.com/github/Imsachin010/ait-511-ms2025013/blob/main/XGBoost_Main1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# data set loading
#
df_train1 = pd.read_csv('/content/drive/MyDrive/MS Research/Sem 1/ML/ait-511-course-project-1-obesity-risk/train.csv')
df_train2 = pd.read_csv('/content/drive/MyDrive/MS Research/Sem 1/ML/ObesityDataSet.csv')
df_test_raw = pd.read_csv('/content/drive/MyDrive/MS Research/Sem 1/ML/ait-511-course-project-1-obesity-risk/test.csv')

print("Data loaded successfully")

Data loaded successfully


In [13]:
df_train2 = df_train2.rename(columns={'NObeyesdad': 'WeightCategory'})
display(df_train2.head())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,WeightCategory
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [14]:
df_train = pd.concat([df_train1, df_train2], ignore_index=True)
print("Training dataframes concatenated successfully.")

# Check for and remove duplicates
initial_rows = df_train.shape[0]
df_train.drop_duplicates(inplace=True)
rows_after_dropping = df_train.shape[0]

print(f"Initial number of rows: {initial_rows}")
print(f"Number of rows after dropping duplicates: {rows_after_dropping}")
print(f"Number of duplicate rows removed: {initial_rows - rows_after_dropping}")

display(df_train.head())

Training dataframes concatenated successfully.
Initial number of rows: 17644
Number of rows after dropping duplicates: 17620
Number of duplicate rows removed: 24


,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,WeightCategory
0,0.0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1.0,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2.0,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3.0,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4.0,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [15]:
df_train.shape

(17620, 18)

In [16]:
X = df_train.drop(['WeightCategory', 'id'], axis=1)
y = df_train['WeightCategory']

## Identify categorical and numerical features



In [17]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

Categorical columns: ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
Numerical columns: ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']


In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

display(X.head())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,2,3
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,3,0
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,3,3
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,2,3
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,2,3


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

display(X.head())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,0.097875,-0.002792,-0.227929,1,1,-0.828762,0.322059,2,0,1.215278,0,-1.169698,0.591656,2,3
1,0,-1.021963,-1.587545,-1.164230,1,1,-0.828762,0.345459,1,0,-0.040994,0,0.022396,0.630666,3,0
2,0,-1.021963,0.126956,-1.423611,1,1,-1.053590,-1.879659,2,0,-0.188445,0,-0.137291,1.747529,3,3
3,0,-0.508758,0.118693,1.654731,1,1,1.053180,0.345459,2,0,-0.577247,0,0.580132,0.266216,2,3
4,1,1.348946,2.421780,0.232371,1,1,0.450326,-1.095437,2,0,-0.074149,0,1.176310,0.517453,2,3


In [21]:
# Calculate correlation with the target variable
# First, we need to encode the target variable as well to calculate correlation
from sklearn.preprocessing import LabelEncoder
y_encoded = LabelEncoder().fit_transform(y)

correlation_with_target = X.corrwith(pd.Series(y_encoded))

# Identify features with low absolute correlation (e.g., less than a threshold like 0.1)
low_corr_features = correlation_with_target[abs(correlation_with_target) < 0.1].index.tolist()

print("Features with low correlation to the target:", low_corr_features)

Features with low correlation to the target: ['Gender', 'Height', 'FAVC', 'FCVC', 'NCP', 'SMOKE', 'SCC', 'TUE', 'MTRANS']


In [22]:
from sklearn.decomposition import PCA

# Apply PCA only to the low correlation features
if low_corr_features:
    X_low_corr = X[low_corr_features]
    pca = PCA(n_components=1) # You can adjust n_components as needed
    X_pca = pca.fit_transform(X_low_corr)

    # Add the PCA component as a new feature to the original DataFrame
    X['PCA_Low_Corr'] = X_pca

    print("PCA applied to low correlation features and added as a new feature.")
    display(X.head())
else:
    print("No features with low correlation found to apply PCA.")

PCA applied to low correlation features and added as a new feature.


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,PCA_Low_Corr
0,1,0.097875,-0.002792,-0.227929,1,1,-0.828762,0.322059,2,0,1.215278,0,-1.169698,0.591656,2,3,0.478104
1,0,-1.021963,-1.587545,-1.164230,1,1,-0.828762,0.345459,1,0,-0.040994,0,0.022396,0.630666,3,0,-1.886058
2,0,-1.021963,0.126956,-1.423611,1,1,-1.053590,-1.879659,2,0,-0.188445,0,-0.137291,1.747529,3,3,0.932213
3,0,-0.508758,0.118693,1.654731,1,1,1.053180,0.345459,2,0,-0.577247,0,0.580132,0.266216,2,3,0.752124
4,1,1.348946,2.421780,0.232371,1,1,0.450326,-1.095437,2,0,-0.074149,0,1.176310,0.517453,2,3,0.282058


In [27]:
# Define X_test by dropping the 'id' column from the raw test data
X_test = df_test_raw.drop('id', axis=1)
display(X_test.head())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,19.007177,1.772449,137.852618,yes,yes,3.000000,3.000000,Sometimes,no,2.007348,no,1.465931,0.813235,Sometimes,Public_Transportation
1,Female,21.572114,1.698346,75.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.577824,1.865851,Sometimes,Public_Transportation
2,Male,22.285024,1.737453,82.000000,yes,yes,2.000000,2.720642,Sometimes,no,1.830614,no,0.451009,0.000000,Sometimes,Public_Transportation
3,Male,30.916426,1.775580,120.860386,yes,yes,2.712747,3.000000,Sometimes,no,2.144368,no,0.712726,0.100048,Sometimes,Automobile
4,Female,18.000000,1.670000,65.000000,no,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.000000,1.000000,no,Public_Transportation


In [28]:
# Apply the same transformation to the test data
if low_corr_features:
    # Ensure categorical columns in test data are encoded
    for col in categorical_cols:
        if col in X_test.columns:
            # Combine unique values from original training data and test data for fitting
            # We need to use the original training data before it was encoded
            combined_unique_values = pd.concat([df_train[col], df_test_raw[col]], axis=0).unique()
            label_encoder.fit(combined_unique_values)
            X_test[col] = label_encoder.transform(X_test[col])

    # Ensure numerical columns in test data are scaled
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

    X_test_low_corr = X_test[low_corr_features]
    X_test_pca = pca.transform(X_test_low_corr)
    X_test['PCA_Low_Corr'] = X_test_pca
    print("PCA transformation applied to low correlation features in test data.")
    display(X_test.head())
else:
    print("No low correlation features were found in training data, so no PCA applied to test data.")

PCA transformation applied to low correlation features in test data.


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,PCA_Low_Corr
0,0,-0.846909,0.817341,1.904378,1,1,1.053180,0.345459,2,0,-0.028905,0,0.577829,0.320992,2,3,0.660400
1,0,-0.401106,-0.021492,-0.481074,1,1,-0.828762,0.345459,2,0,-0.040994,0,0.711216,2.066325,2,3,1.074751
2,1,-0.277198,0.421193,-0.215402,1,1,-0.828762,-0.045902,2,0,-0.319677,0,-0.632053,-1.027425,2,3,-0.115350
3,1,1.222996,0.852784,1.259470,1,1,0.512586,0.345459,2,0,0.196528,0,-0.320062,-0.861536,2,0,-2.662905
4,0,-1.021963,-0.342364,-0.860605,0,1,-0.828762,0.345459,2,0,-0.040994,0,0.022396,0.630666,3,3,0.655020


In [29]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.3 MB/s eta 0:00:00


# Task
Optimize an XGBoost Classifier model using Optuna with StratifiedKFold cross-validation (n_splits=10) on the training data (`df_train`) and define the objective function with the provided parameter search space.

## Import necessary libraries

### Subtask:
Import necessary libraries


**Reasoning**:
Import the necessary libraries for model optimization and evaluation.



In [30]:
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

In [38]:
from sklearn.preprocessing import LabelEncoder

def objective(trial):

    params = {
        'objective': 'multi:softmax',
        'num_class': len(y.unique()),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'random_state': 42,
        'eval_metric': 'mlogloss'
    }

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    f1_scores = []

    # Encode the target variable within the objective function
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    for train_index, val_index in skf.split(X, y_encoded):  # Use y_encoded for splitting
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train_encoded, y_val_encoded = y_encoded[train_index], y_encoded[val_index] # Use encoded y

        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train_encoded) # Fit with encoded y
        y_pred_encoded = model.predict(X_val)

        # Decode the predictions for calculating F1-score with original labels if needed,
        # but f1_score can work with integer labels as well.
        # For consistency with the previous successful step, we'll use encoded y for f1_score
        f1 = f1_score(y_val_encoded, y_pred_encoded, average='weighted')
        f1_scores.append(f1)

    # Return the mean F1-score across all folds
    return np.mean(f1_scores)

In [39]:
# Initialize the Optuna study
study = optuna.create_study(direction='maximize') # We want to maximize F1-score

# Now, retry the optimization
study.optimize(objective, n_trials=100)

# Print the best trial and its value
print("Best trial:")
print(f"  Value: {study.best_trial.value}")
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-10-26 11:09:29,671] A new study created in memory with name: no-name-5b1e7d6d-7c30-429c-8c7d-78c536b8d364
[I 2025-10-26 11:11:04,203] Trial 0 finished with value: 0.9139887182635629 and parameters: {'n_estimators': 402, 'learning_rate': 0.015833693159026472, 'max_depth': 7, 'subsample': 0.5174056970220531, 'colsample_bytree': 0.669763834090384, 'gamma': 0.38762315809449815}. Best is trial 0 with value: 0.9139887182635629.
[I 2025-10-26 11:12:23,298] Trial 1 finished with value: 0.9090218014477582 and parameters: {'n_estimators': 401, 'learning_rate': 0.1394814242610697, 'max_depth': 6, 'subsample': 0.9342656187902305, 'colsample_bytree': 0.9670974934582892, 'gamma': 0.043220301227142766}. Best is trial 0 with value: 0.9139887182635629.
[I 2025-10-26 11:14:20,315] Trial 2 finished with value: 0.9096733370595238 and parameters: {'n_estimators': 401, 'learning_rate': 0.07691900284758975, 'max_depth': 9, 'subsample': 0.5507859901793086, 'colsample_bytree': 0.5866223864984834, 'gamm

KeyboardInterrupt: 

In [40]:
best_params = {'n_estimators': 710, 'learning_rate': 0.010271134167634338, 'max_depth': 8, 'subsample': 0.5021302163410792, 'colsample_bytree': 0.7459406471326316, 'gamma': 0.37584487599752336}


In [73]:
best_params = best_params = {
    "objective": "multi:softmax",
    #"num_class": len(np.unique(y)),
    #"eval_metric": "mlogloss",
    "tree_method": "hist",
    "grow_policy": "lossguide",
    "learning_rate": 0.041,
    "max_depth":  7,
    "min_child_weight": 9,
    "subsample": 0.72,
    "colsample_bytree": 0.61,
    "gamma": 0.68,
    "reg_lambda": 1.045,
    "reg_alpha": 1.468,
    "max_bin": 466,
    "n_estimators": 625,

}

In [74]:
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Encode the target variable globally for use outside the objective function
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


final_refined_model = xgb.XGBClassifier( # Correct the class name
    #objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=len(np.unique(y_encoded)), # Use y_encoded here
    random_state=42,
    **best_params
)

# Train the model on the entire training data
final_refined_model.fit(X, y_encoded) # Use X and y_encoded

print("Final XGBoost model trained with best parameters from Optuna.")


Final XGBoost model trained with best parameters from Optuna.


In [75]:
from sklearn.metrics import accuracy_score

# Make predictions on the training data
y_train_pred_encoded = final_refined_model.predict(X)

# Calculate training accuracy
train_accuracy = accuracy_score(y_encoded, y_train_pred_encoded)

print(f"Training Accuracy of the final model: {train_accuracy:.4f}")

Training Accuracy of the final model: 0.9425


## Make Predictions on Test Data and Create Submission File

In [76]:
# Make predictions on the preprocessed test data
y_test_pred_encoded = final_refined_model.predict(X_test)

# Decode the predictions back to original labels
# Use the same label_encoder that was fitted on the training data
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred_encoded)

# Create a submission DataFrame with 'id' and 'WeightCategory'
submission_df = pd.DataFrame({'id': df_test_raw['id'], 'WeightCategory': y_test_pred_labels})

# Display the first few rows of the submission DataFrame
display(submission_df.head())

# Save the submission file to a CSV
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

,id,WeightCategory
0,15533,Obesity_Type_III
1,15534,Overweight_Level_I
2,15535,Overweight_Level_II
3,15536,Obesity_Type_II
4,15537,Normal_Weight


Submission file 'submission.csv' created successfully.
